In [1]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

In [3]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

In [4]:
dataset = CustomDataset("../datasets/non_linear.csv")
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])
print(f"Training Data Size : {len(train_dataset)}")
print(f"Validation Data Size : {len(validation_dataset)}")
print(f"Testing Data Size : {len(test_dataset)}")

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

Training Data Size : 160
Validation Data Size : 20
Testing Data Size : 20


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [6]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.0993, -1.6991]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.5413], device='cuda:0', requires_grad=True)], Cost : 0.078
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.0989, -1.6994]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.5122], device='cuda:0', requires_grad=True)], Cost : 0.078
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.1003, -1.6992]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.5000], device='cuda:0', requires_grad=True)], Cost : 0.081
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.1003, -1.6994]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4947], device='cuda:0', requires_grad=True)], Cost : 0.077
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.1010, -1.6992]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4925], device='cuda:0', requires_grad=True)]

In [7]:
with torch.no_grad():
    model.eval()
    for x, y in validation_dataloader:
        x = x.to(device)
        y = y.to(device)
        
        outputs = model(x)
        print(f"X : {x}")
        print(f"Y : {y}")
        print(f"Outputs : {outputs}")
        print("--------------------")

X : tensor([[10.8900,  3.3000],
        [25.0000, -5.0000],
        [ 0.4900,  0.7000],
        [31.3600,  5.6000]], device='cuda:0')
Y : tensor([[28.5400],
        [86.2900],
        [ 0.8600],
        [88.2700]], device='cuda:0')
Outputs : tensor([[28.6433],
        [86.4880],
        [ 0.8206],
        [88.1934]], device='cuda:0')
--------------------
X : tensor([[11.5600, -3.4000],
        [53.2900,  7.3000],
        [ 1.4400,  1.2000],
        [17.6400, -4.2000]], device='cuda:0')
Y : tensor([[ 42.4700],
        [152.8700],
        [  2.8300],
        [ 62.6700]], device='cuda:0')
Outputs : tensor([[ 42.1047],
        [153.2890],
        [  2.9160],
        [ 62.3123]], device='cuda:0')
--------------------
X : tensor([[ 1.2100,  1.1000],
        [13.6900, -3.7000],
        [27.0400,  5.2000],
        [34.8100, -5.9000]], device='cuda:0')
Y : tensor([[  2.7600],
        [ 49.6900],
        [ 75.1200],
        [118.7100]], device='cuda:0')
Outputs : tensor([[  2.3729],
        [ 49